## Part 1: Preprocessing

In [66]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

In [67]:
#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [68]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [69]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[["Attrition", "Department"]]

In [70]:
# Create a list of at least 10 column names to use as X data
X_columns = ['Education', 'Age', 'DistanceFromHome', 'JobSatisfaction',
             'OverTime', 'StockOptionLevel', 'WorkLifeBalance', 'YearsAtCompany',
             'YearsSinceLastPromotion', 'NumCompaniesWorked']


# Create X_df using your selected columns
X_df = attrition_df[X_columns]

# Show the data types for X_df
print(X_df.dtypes)

Education                   int64
Age                         int64
DistanceFromHome            int64
JobSatisfaction             int64
OverTime                   object
StockOptionLevel            int64
WorkLifeBalance             int64
YearsAtCompany              int64
YearsSinceLastPromotion     int64
NumCompaniesWorked          int64
dtype: object


In [71]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=1)


In [72]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
X_df["OverTime"].value_counts()

OverTime
No     1054
Yes     416
Name: count, dtype: int64

In [73]:
X_df["OverTime"] = X_df["OverTime"].map({"Yes": 1, "No": 0})
X_train["OverTime"] = X_train["OverTime"].map({"Yes": 1, "No": 0})
X_test["OverTime"] = X_test["OverTime"].map({"Yes": 1, "No": 0})

C:\Users\fsgin\AppData\Local\Temp\ipykernel_22760\1252069082.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_df["OverTime"] = X_df["OverTime"].map({"Yes": 1, "No": 0})


In [74]:
# Create a StandardScaler
X_scaler = StandardScaler()

# Fit the StandardScaler to the training data
X_scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


In [75]:
y_df["Department"].value_counts()

Department
Research & Development    961
Sales                     446
Human Resources            63
Name: count, dtype: int64

In [76]:
# Create a OneHotEncoder for the Department column
department_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
department_encoder.fit(y_train[["Department"]])
department_columns = department_encoder.get_feature_names_out(['Department'])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_encoded_department = pd.DataFrame(department_encoder.transform(y_train[["Department"]]), columns=department_columns)
y_test_encoded_department = pd.DataFrame(department_encoder.transform(y_test[["Department"]]),  columns=department_columns)

In [77]:
# Create a OneHotEncoder for the Attrition column
attrition_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
attrition_encoder.fit(y_train[["Attrition"]])
attrition_columns = attrition_encoder.get_feature_names_out(['Attrition'])

# Create two new variables by applying the encoder
# to the training and testing data

y_train_encoded_attrition = pd.DataFrame(attrition_encoder.transform(y_train[["Attrition"]]), columns=attrition_columns)
y_test_encoded_attrition = pd.DataFrame(attrition_encoder.transform(y_test[["Attrition"]]), columns=attrition_columns)


In [78]:
y_train_encoded_department

,Department_Human Resources,Department_Research & Development,Department_Sales
0,0.0,1.0,0.0
1,0.0,1.0,0.0
2,0.0,0.0,1.0
3,0.0,1.0,0.0
4,0.0,1.0,0.0
...,...,...,...
1097,0.0,1.0,0.0
1098,0.0,1.0,0.0
1099,1.0,0.0,0.0
1100,0.0,0.0,1.0


## Create, Compile, and Train the Model

In [79]:
# Find the number of columns in the X training data
input_dim = len(X_columns)

# Create the input layer
input_layer = layers.Input(shape=(input_dim,), name='input_features')

# Create at least two shared layers
shared1 = layers.Dense(64, activation="relu", name="shared1")(input_layer)
shared2 = layers.Dense(128, activation="relu", name='shared2')(shared1)

In [80]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_dense = layers.Dense(32, activation="relu", name='department_hidden')(shared2)

# Create the output layer
department_output = layers.Dense(
    len(y_train_encoded_department.nunique()),
    activation = 'softmax',
    name = 'department_output'
)(department_dense)


In [82]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_dense = layers.Dense(32, activation="relu", name='attrition_hidden')(shared2)

# Create the output layer
attrition_output = layers.Dense(
    len(y_train_encoded_attrition.nunique()),
    activation = 'sigmoid',
    name = 'attrition_output'
)(attrition_dense)


In [84]:
# Create the model
model = Model(
    inputs = input_layer,
    outputs = [attrition_output, department_output],
)

# Compile the model
model.compile(
    optimizer='adam',
    loss={
        'department_output' : 'categorical_crossentropy',
        'attrition_output' : "binary_crossentropy",
    }
)

# Summarize the model
model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_features      │ (None, 10)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ shared1 (Dense)     │ (None, 64)        │        704 │ input_features[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ shared2 (Dense)     │ (None, 128)       │      8,320 │ shared1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_hidden    │ (None, 32)        │      4,128 │ shared2[0][0]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ department_hidden   │ (None, 32)        │      4,128 │ shared2[0][0]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_output    │ (None, 2)         │         66 │ attrition_hidden… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ department_output   │ (None, 3)         │         99 │ department_hidde… │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 17,445 (68.14 KB)

 Trainable params: 17,445 (68.14 KB)

 Non-trainable params: 0 (0.00 B)

In [85]:
# Train the model
model.fit(
    X_train_scaled,
    {
        'attrition_output' : y_train_encoded_attrition,
        'department_output' : y_train_encoded_department
    },
    epochs=100,
    batch_size=32,
    validation_split=0.2
)

Epoch 1/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - attrition_output_loss: 0.6644 - department_output_loss: 0.9420 - loss: 1.6068 - val_attrition_output_loss: 0.4982 - val_department_output_loss: 0.8427 - val_loss: 1.3422
Epoch 2/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - attrition_output_loss: 0.4474 - department_output_loss: 0.7691 - loss: 1.2168 - val_attrition_output_loss: 0.4862 - val_department_output_loss: 0.8551 - val_loss: 1.3426
Epoch 3/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - attrition_output_loss: 0.4055 - department_output_loss: 0.7663 - loss: 1.1718 - val_attrition_output_loss: 0.4700 - val_department_output_loss: 0.8780 - val_loss: 1.3492
Epoch 4/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - attrition_output_loss: 0.3864 - department_output_loss: 0.7422 - loss: 1.1285 - val_attrition_output_loss: 0.4697 - val_department_output_loss: 0.8577 - val_loss: 1.3287
Epoch 5/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - attrition_output_loss: 0.3310 - department_output_los

In [88]:
# Evaluate the model with the testing data
results = model.evaluate(np.array(X_test_scaled), {
    attrition_output: y_test_encoded_attrition,
    department_output: y_test_encoded_department
})

ValueError: Cannot broadcast shape, the failure dim has value 3, which cannot be broadcasted to 2. Input shapes are: [None, 3] and [None, 2].

In [ ]:
# Print the accuracy for both department and attrition


Department predictions accuracy: 0.5271739363670349
Attrition predictions accuracy: 0.8260869383811951


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1.
2.
3.